# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
# Import dependencies
import azureml.core
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.dataset import Dataset
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.widgets import RunDetails

import joblib 
import numpy as np
import pandas as pd
import os

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


## Initialize Workspace & Create Experiment

Initialize workspace from config, then create experiment within the workspace

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-137967
aml-quickstarts-137967
southcentralus
510b94ba-e453-4417-988b-fbdc37b55ca7


In [3]:
experiment_name = 'hyperdrive-heart-failure'
project_folder = './heartfailure'

experiment = Experiment(ws, experiment_name)

## Create or Attach an Auto ML Compute Cluster
Search for existing compute cluster; if not found, create one

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# If using a different existing cluster, enter the name in place of 'mlecscompute' below:
amlcompute_cluster_name = 'mlecpcompute'

# Verify cluster existence: 
try: 
    compute_target = ComputeTarget(workspace = ws, name = amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Not found; creating new cluster.')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_v2', max_nodes=4, min_nodes=1)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output = True)

Found existing cluster, use it.

Running


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
# Attempt to load the dataset from the Workspace, but otherwise, create from file
# dataset file located at 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
found = False
# key should be set to 'heartfailuredataset' 
key = 'heartfailuredataset'
# description_text should be set to 'heartfailuredataset' 
description_text = 'heartfailuredataset'

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]

if not found:
    datasetfile = 'https://raw.githubusercontent.com/RachelAnnDrury/MLECapstone/main/heart_failure_clinical_records_dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(datasetfile)
    dataset = dataset.register(workspace = ws, name = key, description = description_text)

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
# HyperDrive Early Stopping Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1)

# HyperDrive Parameter Sampler
ps = RandomParameterSampling(
    {
        '--C' : uniform(0, 1),
        '--max_iter' : choice(10, 50, 100, 120, 200)
    }
)

#TODO: Create your estimator and hyperdrive config
if "training" not in os.listdir():
    os.mkdir("./training")

# Pass parameters to training script and create HyperDriveConfig
est = SKLearn(source_directory = '.', 
    entry_script = 'train.py',
    compute_target = compute_target)

hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    primary_metric_name = 'Accuracy',
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 100,
    max_concurrent_runs = 4,
    policy = policy,
    estimator = est
)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
# Submit Hyperdrive Experiment
hyperdrive_run = experiment.submit(config=hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
# Run Details in Azure Widget
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
# TODO: Save the best model:
# This is copied from the Azure Hyperlight notebook; some formatting updated
best_run = hyperdrive_run.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(parameter_values)

In [ ]:
#TODO: Save the best model
# This is copied from the Azure Hyperlight notebook; model name has been updated
model = best_run.register_model(
    model_name = 'hyperheartmodel', model_path = 'outputs/model'
)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service